# Implementation of the K-Nearest Neighbors Algorithm on Movie Data Set

### by Laura Moses

How do you choose your k? see vid.. 


Advantages 
* Simple
* No optimization of parameters
Disadvantages
* Slow
* Sensitive to high dimension feature vector

Better in conjunction with Principal Component Analysis (PCA)

$$d(x^i, x^j) = \sqrt{\Sigma_{n=1}^l(x_n^i - x_n^j)^2}$$

In [3]:
# import/load necessary packages
#import Pkg; Pkg.add("RDatasets")
#import Pkg; Pkg.add("Plots")
#import Pkg; Pkg.add("CSV")
using RDatasets
using Plots
using CSV

┌ Info: Precompiling RDatasets [ce6b1742-4840-55fa-b093-852dadbb1d8b]
└ @ Base loading.jl:1278
┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1278


In [6]:
iris = dataset("datasets", "iris")

x_data = [x for x in zip(iris.SepalLength[1:150], iris.SepalWidth[1:150, iris.PetalLength])]

y_data = [x for x in iris.Species[x]]

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
,Float64,Float64,Float64,Float64,Cat…
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa
6,5.4,3.9,1.7,0.4,setosa
7,4.6,3.4,1.4,0.3,setosa
8,5.0,3.4,1.5,0.2,setosa
9,4.4,2.9,1.4,0.2,setosa


LoadError: UndefVarError: data not defined

Movies example

In [ ]:
movies = CSV.read("movies_recommendation_data", DataFrame)

In [ ]:
x_movie_data = [x for in zip(movies.IMDBRating, movies.Biography, movies.Drama, movies.Thriller)]
y_movie_data = [x for x in movies.MovieName]

In [ ]:
function more_like_this(movie_name, features, labels, k)
    for i = 1:length(labels)
        if labels[i] == movie_name
            neighbors = KNN(features[i], fetures, labels, k+1)
            printlin("The top $k similar movies with $movie_name are:")
            for j = 2:k+1
                println("$(j-1), ", neighbors[j][2])
            end
        end
    end
end


In [ ]:
more_like_this("The Terminator", x_movie_data, y_movie_data, 7)